In [2]:
import jieba
import pdfplumber
import json

In [4]:
questions = json.load(open('./questions.json'))

In [5]:
pdf = pdfplumber.open('./初赛训练数据集.pdf')

In [8]:
pdf_content = []

for idx, content in enumerate(pdf.pages):
    pdf_content.append(
        {
            'page_idx': str(idx + 1),
            'page_content': content.extract_text()
        }
    )

In [9]:
pdf_content[189]

{'page_idx': '190',
 'page_content': '驾驶辅助\n警告！ 停用高级智能驾驶\n■ 设置的跟车距离间隔较低，可能会导致发生意外情况时来不及反\n应。\n■ 您始终有责任始终保持安全的距离和速度，并在必要时及时采取\n制动措施。\n说明！\n□ 跟车距离间隔不是固定距离，与前车之间的实际距离会随车速变\n化而变化。\n□ 在湿滑路面行驶时，请选择更大的跟车距离间隔。\n□ 踩下加速踏板时，车辆将暂时加速；松开加速踏板后，车速将缓\n慢减速至目标车速。\n□ 若您开启了“巡航自动限速辅助”功能，系统可以将当前道路限 按下方向盘上的 按键或者踩下制动踏板，暂时停用LynkCo-Pilot功\n速牌上的限速值自动设定为目标车速。\n能， 图标变为白色，LynkCo-Pilot进入准备模式。\n190'}

In [10]:
import torch
import transformers

In [11]:
from transformers import  AutoTokenizer, AutoModelForSequenceClassification

In [12]:
tokenizer = AutoTokenizer.from_pretrained('../BAAI/bge-reranker-base/')

In [13]:
rerank_model = AutoModelForSequenceClassification.from_pretrained('../BAAI/bge-reranker-base/')

In [14]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [15]:
device

device(type='cuda', index=0)

In [16]:
rerank_model.to(device)

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [19]:
pdf_content_words = [jieba.lcut(x['page_content']) for x in pdf_content]

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ASUS\AppData\Local\Temp\jieba.cache
Loading model cost 0.640 seconds.
Prefix dict has been built successfully.


In [20]:
from rank_bm25 import  BM25Okapi

In [21]:
bm25 = BM25Okapi(pdf_content_words)

In [24]:
for query_idx in range(len(questions)):
    page_scores = bm25.get_scores(jieba.lcut(questions[query_idx]['question']))
    max_score_page_indexes = page_scores.argsort()[-3:]

    pairs = [[questions[query_idx]['question'], pdf_content[page_idx]['page_content']] for page_idx in max_score_page_indexes]

    inputs = tokenizer(pairs, padding=True, truncation=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in inputs.items()}
        scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()
    
    best_match_index = max_score_page_indexes[scores.cpu().numpy().argmax()]
    questions[query_idx]['reference'] = 'page_' + str(best_match_index + 1)


with open('./submit_3.json', 'w', encoding='utf8') as f:
    json.dump(questions, f, ensure_ascii=False, indent=4)

